In [36]:
import numpy as np 
from scipy.optimize import fsolve

## Adiabatic model

In [21]:
R = 8.314
pi = np.pi
f_prime_guess = 0.02


In [44]:
P1 = 
T1 = 
M = 
mu1 = 
W = 
D1 = 
L1 = 
epsilon = 
Cp1 = 
Cv1 = 

SyntaxError: invalid syntax (237607588.py, line 1)

In [32]:
def f_prime(x,epsilon,D,Re):
    LHS=1/np.sqrt(x)
    RHS=-2*np.log10(e/D/3.7+2.51/Re/np.sqrt(x))
    return LHS - RHS

In [33]:
def rho(rho2, f_prime, L, D, k, rho1, P1, A, W):
    LHS = f_prime*L/D
    RHS = ((k+1)/k)*np.log(rho2/rho1) + (1-(rho2/rho1)**2)*((k-1)/(2*k)+P1*rho1*(A/W)**2)
    return LHS - RHS

In [34]:
def P(P2, W, A, k, P1, rho1, rho2):
    LHS = (W/(rho1*A))**2 + 2*k*(P1/rho1)/(k-1)
    RHS = (W/(rho2*A))**2 + 2*k*(P2/rho2)/(k-1)
    return LHS - RHS

In [15]:
rho1 = M*P1/(R*T1)

A1 = pi*D1**2/4

k1 = Cp1/Cv1

u1 = W/(rho1*A)

Re1 = rho1*u1*D/(mu1**2)

f_prime_1 = fsolve(f_prime, f_prime_guess, args=(epsilon,D1,Re1))[0]

rho2 = fsolve(rho, rho1, args=(f_prime, L1, D1, k, rho1, P1, A, W))

NameError: name 'M' is not defined